In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
import pylatexenc


def compute_response(content):
    n = content['n']
    m = content['m']
    gates = content['gates']
    input_states = content['cats']
    output = content['output']
    
    circuit = Circuit(n, m, gates, input_states, output)
    return circuit.run()
    

def place_quantum_gate(gate, qc, row):
    
    if gate == 'x':
        qc.x(row)
    elif gate == 'y':
        qc.y(row)
    elif gate == 'z':
        qc.z(row)
    elif gate == 'h':
        qc.h(row)
    elif gate == 'i':
        qc.i(row)


def place_controlled_quantum_gate(controlled_qubit, conditional_qubit, gate, qc):
    print('controlled: ', controlled_qubit)
    print('conditional: ', conditional_qubit)
    
    if gate == 'x':
        qc.cx(controlled_qubit, conditional_qubit)
    elif gate == 'y':
        qc.cy((controlled_qubit, conditional_qubit))
    elif gate == 'z':
        qc.cz((controlled_qubit, conditional_qubit))
    elif gate == 'h':
        qc.ch((controlled_qubit, conditional_qubit))
    elif gate == 'i':
        qc.ci((controlled_qubit, conditional_qubit))

        
def prepare_input_state(qc, qr, input_string):
    for i, bit in enumerate(input_string):
        if bit == '1':
            qc.x(qr[i])
    return qc
        

def handle_results(qc, qr, cr):
    qc.measure(qr, cr)

    simulator = Aer.get_backend('qasm_simulator')
    job = execute(qc, simulator, shots=1000)

    result = job.result()
    counts = result.get_counts(qc)

    print(counts)

    print(qc)
    qc.draw('mpl')
    
    return json.dumps(counts)
        

class Circuit:
    def __init__(self, n, m, gates_matrix, input_states, output_states):
        self.n = n
        self.m = m
        self.gates_matrix = gates_matrix
        self.input_states = input_states
        self.output_states = output_states

    def run(self):
        qr = QuantumRegister(self.n)
        cr = ClassicalRegister(self.n)
        qc = QuantumCircuit(qr, cr)
        
        prepare_input_state(qc, qr, self.input_states)

        for row in range(self.n):
            for column in range(self.m):
                gate_info = self.gates_matrix[row][column]
                
                if gate_info:
                
                    is_controlled, info = gate_info

                    if is_controlled:
                        controlled_qubit, conditional_qubit, gate = info
                        place_controlled_quantum_gate(controlled_qubit, conditional_qubit, gate, qc)
                    else:
                        place_quantum_gate(info, qc, row)

        return handle_results(qc, qr, cr)


In [2]:
# Python 3 server example
from http.server import BaseHTTPRequestHandler, HTTPServer
import time

import json

hostName = "localhost"
serverPort = 8082

class MyServer(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(bytes(json.dumps(a, ensure_ascii=False), 'utf-8'))
    
    def do_POST(self):
        content_len = int(self.headers.get('Content-Length'))
        post_body = self.rfile.read(content_len)
        
        content = json.loads(post_body)
        content_dict = json.loads(content)
        print('Request with body: ', content_dict)
        
        self.send_response(200)
        self.send_header('Content-type','text/html')
        self.end_headers()

        message = compute_response(content_dict)
        self.wfile.write(bytes(message, "utf8"))
        
if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        pass

    webServer.server_close()
    print("Server stopped.")

Server started http://localhost:8082
Request with body:  {'n': 2, 'm': 2, 'gates': [[[False, 'x'], [True, [1, 1, 'x']]], [[False, 'x'], [False, 'x']]], 'cats': '11', 'output': [0, 0]}
controlled:  1
conditional:  1


127.0.0.1 - - [22/Jun/2023 17:49:15] "POST / HTTP/1.1" 200 -
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 60137)
Traceback (most recent call last):
  File "D:\AnacondaPython\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "D:\AnacondaPython\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "D:\AnacondaPython\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "D:\AnacondaPython\lib\socketserver.py", line 747, in __init__
    self.handle()
  File "D:\AnacondaPython\lib\http\server.py", line 427, in handle
    self.handle_one_request()
  File "D:\AnacondaPython\lib\http\server.py", line 415, in handle_one_request
    method()
  File "C:\Users\Delia\AppData\Local\Temp/ipykernel_15428/3045681239.py", line 29, in do_POST
    message = compute

Request with body:  {'n': 2, 'm': 2, 'gates': [[[False, 'x'], [True, [1, 1, 'x']]], [[False, 'x'], [False, 'x']]], 'cats': '00', 'output': [2, 3]}
controlled:  1
conditional:  1


127.0.0.1 - - [22/Jun/2023 17:52:17] "POST / HTTP/1.1" 200 -
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 60203)
Traceback (most recent call last):
  File "D:\AnacondaPython\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "D:\AnacondaPython\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "D:\AnacondaPython\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "D:\AnacondaPython\lib\socketserver.py", line 747, in __init__
    self.handle()
  File "D:\AnacondaPython\lib\http\server.py", line 427, in handle
    self.handle_one_request()
  File "D:\AnacondaPython\lib\http\server.py", line 415, in handle_one_request
    method()
  File "C:\Users\Delia\AppData\Local\Temp/ipykernel_15428/3045681239.py", line 29, in do_POST
    message = compute

Request with body:  {'n': 2, 'm': 2, 'gates': [[[True, [0, 1, 'x']], [False, 'x']], [[False, 'x'], [False, 'x']]], 'cats': '00', 'output': [2, 3]}
controlled:  0
conditional:  1


...

Request with body:  {'n': 2, 'm': 2, 'gates': [[[True, [0, 1, 'x']], [False, 'x']], [[False, 'x'], [False, 'x']]], 'cats': '00', 'output': [2, 3]}
controlled:  0
conditional:  1
{'01': 1000}
             ┌───┐     ┌─┐   
q108_0: ──■──┤ X ├─────┤M├───
        ┌─┴─┐├───┤┌───┐└╥┘┌─┐
q108_1: ┤ X ├┤ X ├┤ X ├─╫─┤M├
        └───┘└───┘└───┘ ║ └╥┘
 c17: 2/════════════════╩══╩═
                        0  1 


127.0.0.1 - - [22/Jun/2023 18:33:15] "POST / HTTP/1.1" 200 -


Request with body:  {'n': 4, 'm': 4, 'gates': [[[False, 'x'], [False, 'x'], [False, 'x'], [False, 'x']], [[False, 'x'], [False, 'x'], [False, 'h'], [False, 'h']], [[False, 'x'], [False, 'x'], [False, 'x'], [False, 'h']], [[False, 'h'], [False, 'x'], [True, [3, 0, 'x']], [False, 'x']]], 'cats': '0000', 'output': [2, 3, 3, 3]}
controlled:  3
conditional:  0
{'1000': 270, '0101': 244, '0001': 234, '1100': 252}
        ┌───┐┌───┐┌───┐┌───┐┌───┐           ┌─┐   
q123_0: ┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├───────────┤M├───
        ├───┤├───┤├───┤├───┤└─┬─┘┌─┐        └╥┘   
q123_1: ┤ X ├┤ X ├┤ H ├┤ H ├──┼──┤M├─────────╫────
        ├───┤├───┤├───┤├───┤  │  └╥┘┌─┐      ║    
q123_2: ┤ X ├┤ X ├┤ X ├┤ H ├──┼───╫─┤M├──────╫────
        ├───┤├───┤└───┘└───┘  │   ║ └╥┘┌───┐ ║ ┌─┐
q123_3: ┤ H ├┤ X ├────────────■───╫──╫─┤ X ├─╫─┤M├
        └───┘└───┘                ║  ║ └───┘ ║ └╥┘
 c18: 4/══════════════════════════╩══╩═══════╩══╩═
                                  1  2       0  3 


127.0.0.1 - - [22/Jun/2023 18:33:45] "POST / HTTP/1.1" 200 -


Server stopped.
